In [1]:
! pip3 install selenium
! pip3 install beautifulsoup4

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time

In [3]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_path = "/Users/leo/Desktop/Analytics/2020 Fall/ise540/PROJECT_1116due/爬蟲/chromedriver"

In [5]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [6]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        print("find xpath timeout:",xpath)

In [11]:
comment_info_xpath = "//div[@id='main']"
show_more_button_xpath = "//span[@class='more-button style-scope ytd-comment-renderer']"
# comment_total_amount_xpath = "//yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"

In [23]:
all_urls = [
    'https://www.youtube.com/watch?v=EPoOBF8-Tvs&ab_channel=NosIsBack',
    'https://www.youtube.com/watch?v=vi7SeBI7z9A&ab_channel=ArseRaptor'
#     , 'https://www.youtube.com/watch?v=KxoktuehP3c&ab_channel=TrevorNoah'
#     , 'https://www.youtube.com/watch?v=30cYjy7_DDg&ab_channel=NetflixIsAJoke'
#     , 'https://www.youtube.com/watch?v=nNx5tnqD9e0&ab_channel=JustForLaughs'
]
all_urls = collections.deque(all_urls)

In [24]:
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:
    web_list = []
    while all_urls:
        prev_comments_amount = -1
        all_comments_info = []
        driver.get(all_urls.popleft())
    ##### keep scrolling, until we get the same amount of comments of comments_total_amount
        while len(all_comments_info) != prev_comments_amount:
            print(prev_comments_amount, len(all_comments_info))
            prev_comments_amount = len(all_comments_info)
            driver.execute_script("scrollBy(0, 100000)")
    #         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #window.scrollTo(0，Y),Y是高度,document.body.scrollHeight→滾動到無限加載的頁面
            time.sleep(5)
            all_comments_info = driver.wait_elements_xpath(comment_info_xpath)
            print(len(all_comments_info)) # count how many comments were scraped so far
        html = driver.page_source     # Get whole html for us to find sth later
        whole_page = bs(html,"html.parser")
        web_list.append(whole_page)

-1 0
20
0 20
41
20 41
61
41 61
81
61 81
101
81 101
109
101 109
109
-1 0
20
0 20
40
20 40
60
40 60
80
60 80
100
80 100
120
100 120
140
120 140
160
140 160
180
160 180
200
180 200
220
200 220
240
220 240
260
240 260
280
260 280
300
280 300
320
300 320
340
320 340
360
340 360
380
360 380
400
380 400
420
400 420
440
420 440
447
440 447
447


In [26]:
len(web_list)

2

In [17]:
# comments = whole_page.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
# comments_names_list = []
# comments_contents_list = []
# comments_thumbsup_list = []

# for comment in comments:
#     # reviewer's name
#     name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()
#     comments_names_list.append(name.strip('\n '))
#     # reviewer's comment
#     content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
#     comments_contents_list.append(content)
#     # reviewer's thumbsup amount
#     thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
#     new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
#     comments_thumbsup_list.append(int(eval(new_thumbsup)))
    
# print(len(comments_names_list))
# print(len(comments_contents_list))
# print(len(comments_thumbsup_list))

In [31]:
def html_to_list(web):
    whole_page = web
    comments = whole_page.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
    comments_names_list = []
    comments_contents_list = []
    comments_thumbsup_list = []

    for comment in comments:
        # reviewer's name
        name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()
        comments_names_list.append(name.strip('\n '))
        # reviewer's comment
        content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
        comments_contents_list.append(content)
        # reviewer's thumbsup amount
        thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
        new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
        comments_thumbsup_list.append(int(eval(new_thumbsup)))

    # list to df
    row = [comments_thumbsup_list, comments_names_list, comments_contents_list]
    df = pd.DataFrame(row).transpose()
    df.columns = ['Thumbsup', 'Name','Comment']
    return df

In [32]:
df = html_to_list(web_list[0])
df

,Thumbsup,Name,Comment
0,1200,Gary Bush,The way he explains his stories are pure comed...
1,428,Emerald 919,I mean cheating isn’t cool but this hilarious
2,658,ShadowIn TheDark,0:35 explains Kevin Hart in 1 Breath
3,524,Bebe Ballerina,damn i hate that he cheated sm but this is so ...
4,229,David Mbagu,Here after reading his book. It never occurred...
5,134,Aze Edits,1:32 and thus the meme was born.
6,62,locaoa,mans cheated on his first wife made jokes of i...
7,196,Alfro Samuraii,And then cheated again😭😭
8,73,Professor Blackwood,"Couldn't he just tell his friend, Harry, that ..."
9,46,Darian Canada,2:00 every girl has said this to their man whe...


In [33]:
df = html_to_list(web_list[1])
df

,Thumbsup,Name,Comment
0,4300,Muddy Council,His mastery of accents is underated
1,3500,Legend Rid,“I felt like a bag of weed”😂😂😂😂💀
2,2100,Dire Girl,"“18 hours of flight I wasn’t black, I was Mexi..."
3,1700,Sharad Saraswat,Apartheid! Such a sad reality of life and narr...
4,2000,Sîhêm Nôûr-Elhouda,That hitler part killed me x'D ! Trevor is ama...
5,971,mysterychemistry,"""You know how the Swiss love chocolates"" :D"
6,2400,Mad9977 Productions,"""i was born a crime"" 😂"
7,838,Pete Roz,"""der schwarze hitler"" BUAHAHA - got me in tear..."
8,494,Anna Mouse,No swearing and yet so funny. Thank you.
9,582,Paff,Wow! His German is really good!!!
